<image src="https://camo.githubusercontent.com/14db3cb92f9b9ed943e33ae9a3a9ebe2ae35c9595393ff14e9595be6f8fb140e/68747470733a2f2f692e696d6775722e636f6d2f46534d324e4e562e706e67" width="200" align="center">


# Working with Aleph Alpha Technology
Hi, great to see you working with Aleph Alpha Technology. 

This notebook will support you on your journey by providing you with some examples on how to use our API and how to solve tasks with it.
We will be using the Aleph Alpha API to solve tasks all kinds of tasks and learn how different components and functionalities of our LLMs can be used to solve them.

If you have any questions or feedback, please feel free to reach out to us at [Aleph Alpha support](mailto:support@aleph-alpha.com).

Good luck and have fun!


----------------------

## Prerequisites:
- You have an Aleph Alpha account and API key (you can sign up here: https://app.aleph-alpha.com/)
- You have glanced over our documentation (https://docs.aleph-alpha.com/)
- YOu have played around with our playground (https://app.aleph-alpha.com/playground)


## Content

This notebook will contain information on the following topics:
1. Use our LLMs to generate text and solve tasks
2. Using embeddings to find similar relevant information
3. Use semantic embeddings and completion to answer questions
4. Chaining multiple requests to solve complex tasks
5. Using Atman to explain your results

Let's get started!

### Install dependencies

In [ ]:
%%capture
# Setting up the workspace on colab
!git clone https://github.com/Aleph-Alpha/bootcamp.git
!pip install -r bootcamp/requirements.txt
!cp bootcamp/data.md data.md



##### These are just some imports to start working with our API
If you are interested, here is what the individual imports do:

| Import | Description |
| --- | --- |
| ``Client`` | This is the main class that you will use to authenticate with the API. |
| ``Prompt`` | We use this class to format information correctly for our models |
| ``CompletionRequest`` | CompletionRequests are used to reuqest our models to generate text, e.g. for solving tasks |
| ``SemanticEmbeddingRequest`` | SemanticEmbeddingRequests are used to request our models to generate embeddings for text, e.g. for searching for information or for classification |
| ``ExplanationRequest`` | ExplanationRequests are used to request our models to generate explanations for text, e.g. for explaining a where an answer comes from |
| ``TextControl`` | TextControl allows us to manipulate the attention of our models, e.g. to focus on certain parts of the input |


In [ ]:
from aleph_alpha_client import Client, Prompt, CompletionRequest, CompletionResponse, SemanticEmbeddingRequest, SemanticEmbeddingResponse, SemanticRepresentation, ExplanationRequest, TextControl
from scipy import spatial
import numpy as np
import json
import os

### Step 0: Using the client to authenticate with the API
First, we need to authenticate with the API. To do this, we need to create a ``Client`` object and pass it our API key. You can create your API key in your [account settings](https://app.aleph-alpha.com/profile).

If you want to use the local API, you need to also pass the ``host`` parameter to the client.

In [ ]:
# Authenticate with the API by using the client class
client = Client(token="<your token here>")

# Step 1: Using LLMs to generate text and solve tasks
In this section, we will use our LLMs to generate text and solve tasks.

We will use the same LLM for both tasks. This is because our LLMs are trained to solve many different tasks. This means that we can use the same LLM for many different tasks.

<image src="https://github.com/Aleph-Alpha/bootcamp/blob/main/img/functionalities.png?raw=true" width="800" align="center" alt="Aleph Alpha Functionalities">


We will use the completion endpoint to generate text and solve tasks. You can find more information about this endpoint in the [Completion Documentation](https://docs.aleph-alpha.com/docs/tasks/complete/).

With completions we prompt the model to generate text. Depending on the prompt, the model will generate different text. This is a very powerful universal tool to generate text and solve tasks.

However, to get the best results, we need to formulate our prompts correctly. We need to keep in mind the structure that the model expects and also how to word our requests so that the model understands what we want.



### 1.1 Generating text
First, let's just start with generating text. While our API offers different models, we will start with our ``Control-models``. These models are specifically optimized to solve tasks that you give them.

We will stick to the structure that these models expect. This is a good starting point to get familiar with the API.

```markdown
### Instruction:
INPUT YOUR INSTRUCTION HERE

### Input:
YOUR INPUT (Optional)

### Response:
```

Try to vary the input and see how the model responds. You can also try to change the instruction and see how the model responds.

In [ ]:
# Write a prompt, so that the model knows what to do
prompt_text = """### Instruction: 
What options are there for resetting a password?

### Response:"""

# Create the completion request
request = CompletionRequest(
    prompt=Prompt.from_text(prompt_text), 
    maximum_tokens=20, # Parameter to control the maximum length of the completion
    temperature=0.0, # Parameter to control the randomness of the completion
    stop_sequences=["\n"]) # Parameter to control the stopping criteria of the completion

# Send the prompt to the API
response = client.complete(request=request, model="luminous-base-control")

response_text = response.completions[0].completion

# Print the response
print(f"The model returned: `{response_text}`")

Great, so now we have generated some text. 

However, as you can see this information is coming from the model's foundatinal knowledge. This is not very useful in many cases.

Let's try again, but this time, we provide some background information. This will help the model to generate more useful information.


In [ ]:
# Write a prompt, so that the model knows what to do
prompt_text = """### Instruction: 
What options are there for resetting a password?

### Input:
If a customer forgot their password, they should be able to reset it by clicking on the "Forgot Password" link on the login page.
They can use different methods to reset their password, such as email, SMS, or security questions.

### Response:"""

# Create the completion request
request = CompletionRequest(
    prompt=Prompt.from_text(prompt_text), 
    maximum_tokens=20, # Parameter to control the maximum length of the completion
    temperature=0.0, # Parameter to control the randomness of the completion
    stop_sequences=["\n"]) # Parameter to control the stopping criteria of the completion

# Send the prompt to the API
response = client.complete(request=request, model="luminous-base-control")

response_text = response.completions[0].completion

# Print the response
print(f"The model returned: `{response_text}`")

### 1.2 Solving specific tasks
Now that we have seen how to generate text, let's try to solve a specific task. We will use the same model as before, but we will give it a different instruction.

This time, we want to create a product text for a new product. We will give the model a short description of the product and ask it to generate a product text.

We will be using both the ``Control-models`` as well as the ``foundation-models``. The ``foundation-models`` are trained on a large amount of data and are able to generate text that is more fluent and coherent. However, they are not as good at solving specific tasks as the ``Control-models``.

While control models work with a specific structure, the foundation models are more flexible. This means that we can use them to generate text in a more natural way. However, they require a ``few-shot`` prompt. This means that we need to give them a few examples of what we want them to do.

In [ ]:
# Here is a control model prompt for you to try out. In this case, the model is asked to generate a product description for a yoga mat.
control_prompt_text = """### Instruction:
Generate a product description for the following product.
Only use information from the product description.

### Input:
Name: Multifunctional Yoga Mat
Color: Blue
Material: Rubber
Size: 180 x 60 x 0.5 cm
Uses: Yoga, Pilates, Fitness, Gymnastics, Camping, Picnic, Sleep, Play, etc.

### Response:"""

# Let's send the prompt to the API and see what the model returns
request = CompletionRequest(
    prompt=Prompt.from_text(control_prompt_text),
    maximum_tokens=100,
    temperature=0.0,
    stop_sequences=[])

response = client.complete(request=request, model="luminous-base-control")

response_text = response.completions[0].completion

print(f"The model returned: `{response_text}`")

While this is more flexible, it also means that we need to be more careful with how we formulate our requests. We need to make sure that we give the model enough information to understand what we want it to do. 

With few-shot we can more easily steer the model to generate the text that we want. However, we need to be careful to not give the model useful examples, so that we do not bias it.

In [ ]:
# This is how we would write the prompt as a few-shot learning prompt
few_shot_prompt_text = """Task: Generate a product description for the following product.
Only use information from the product description.
###
Product:
- Name: Ergonomic Office Chair
- Color: Black
- Material: Plastic, Metal, Fabric
- Functions: Height adjustable, 360 degree swivel, seat tilt, back tilt
- Uses: Office, Home, Gaming, etc.
Description: This ergonomic office chair is made of high-quality materials, such as plastic, metal, and fabric and is very comfortable to sit on. 
It is height adjustable, can swivel 360 degrees, and has a seat and back tilt. 
You can use it in the office, at home, or for gaming.
###
Product:
- Name: Multifunctional Yoga Mat
- Color: Blue
- Material: Rubber
- Size: 180 x 60 x 0.5 cm
- Uses: Yoga, Pilates, Fitness, Gymnastics, Camping, Picnic, Sleep, Play, etc.
Description:"""

# Let's send the prompt to the API and see what the model returns
request = CompletionRequest(
    prompt=Prompt.from_text(few_shot_prompt_text),
    maximum_tokens=100,
    temperature=0.5, # We can use a higher temperature to make the model more creative
    stop_sequences=["###"] # with the foundation models we need to specify the stop sequence
    )

response = client.complete(request=request, model="luminous-extended")


response_text = response.completions[0].completion

print(f"The model returned: `{response_text}`")


### 1.3 Experiment with completions LLMs yourself

Now you can go ahead and experiment with completions yourself. 

Try to solve different tasks with the LLMs. 

Experiment with ``Control-models`` and ``foundation-models``. 
See how they differ in their responses and how they solve tasks.

In [ ]:
# TODO Change the prompt to be solve a different task
control_prompt_text = """Try to write your own prompt here."""

# Send the prompt to the API and see what the model returns
request = CompletionRequest(    
    prompt=Prompt.from_text(control_prompt_text),
    maximum_tokens=100,
    temperature=0.0,
    stop_sequences=[])

response = client.complete(request=request, model="luminous-base-control")
response_text = response.completions[0].completion

print(f"The model returned: `{response_text}`")

### Step 1.4: Smalltalk and QA-Chat prompt
We can also use the LLMs to chat with them. This is a fun way to get to know the LLMs and to see how they work.

Write two functions, one for small talk and one for a more specific conversation.
The small talk function should be able to chat about different topics, while the specific conversation should be able to answer specific questions based on a given context.

In [ ]:
# Smalltalk function
def smalltalk(message, history):
    
    history_str = "\n".join(history)
    
    prompt = f"""### Instruction:
You are a friendly chatbot called AlphaBot and developed by Aleph Alpha.
Have a nice and friendly conversation with the user.

### Input:
history:
{history_str}

User: {message}

### Response:
AlphaBot:"""
    
    request = CompletionRequest(
        prompt=Prompt.from_text(prompt),
        maximum_tokens=100,
        temperature=0.3,
        stop_sequences=["\n", "User", "AlphaBot"])

    response = client.complete(request=request, model="luminous-extended-control")
    response_text = response.completions[0].completion

    return response_text

# QA function
def qa_answer(message, history, context):

    history_str = "\n".join(history)
    
    prompt = f"""### Instruction:
You are a friendly chatbot called AlphaBot and developed by Aleph Alpha.
Answer the user's question based on the context.

### Input:
context: {context}

history:
{history_str}

User: {message}

### Response:
Bot:"""


    request = CompletionRequest(
        prompt=Prompt.from_text(prompt),
        maximum_tokens=100,
        temperature=0.0,
        stop_sequences=["\n", "User", "AlphaBot"])

    response = client.complete(request=request, model="luminous-extended-control")
    response_text = response.completions[0].completion

    return response_text

--------------------

### Step 2: Using Embeddings to search for information
In many cases, the relevant information to solve a task may not be available or known to the model.

With Semantic Search, we can use the embeddings to search for relevant information in a corpus of documents. The idea is that LLMs are able to understand the meaning of a question and the meaning of a document, and thus, can find the most relevant document to answer the question.

We do this by first encoding the question and the documents into embeddings. Then, we compute the similarity between the question embedding and the document embeddings. Finally, we return the document with the highest similarity score.

<div>
<img src="https://docs.aleph-alpha.com/assets/images/symmetric_embedding-fdb53a9755c451641d70d08b8f58db8b.png" width="500"/>
</div>

<div>
<img src="https://docs.aleph-alpha.com/assets/images/asymmetric_embedding-6cac7874ae7db8b2cd796bfd2d2f1bcb.png" width="500"/>
</div>

You can find more information about this technique in the [Semantic Embedding Documentation](https://docs.aleph-alpha.com/docs/tasks/semantic_embed/).

Let's see how this works in practice.

### 2.1 Creating embeddings for text.
In Order to find the correct documents, we need to turn our text into numbers.
We do that with semnatic embeddings. These are vectors that represent the meaning of the data.

Let's use Aleph Alpha technology to create embeddings for our text.

In [ ]:
# Two texts and a question to be embedded and searched for
text_1 = """With our semantic_embed-endpoint you can create semantic embeddings for your text. 
This functionality can be used in a myriad of ways. 
For more information please check out our blog-post on Luminous-Explore, introducing the model behind the semantic_embed-endpoint. 
In order to effectively search through your own documents, it is important to ensure that they can be easily compared to each other. 
Our asymmetric embeddings are designed to help find the pieces of your documents that are most relevant to a query shorter than the documents in the database. 
Here we will use short queries and longer splits of law texts."""

text_2 = """You can interact with a Luminous model by sending it a text. We call this a prompt. 
It will then produce text that continues your input and return it to you. This is what we call a completion. 
Generally speaking, our models attempt to find the best continuation for a given input. 
Practically, this means that the model first recognizes the style of the prompt and then attempts to continue it accordingly."""

question = "How can I search through my documents with embeddings?"

In [ ]:
# using the API to embed the text

# We embed the texts as Documents, as the contain a lot of information
request_1 = SemanticEmbeddingRequest(prompt=Prompt.from_text(text_1), representation=SemanticRepresentation.Document)
request_2 = SemanticEmbeddingRequest(prompt=Prompt.from_text(text_2), representation=SemanticRepresentation.Document)

# We embed the question as a Query, as it is a short text
request_question = SemanticEmbeddingRequest(prompt=Prompt.from_text(question), representation=SemanticRepresentation.Query)

# We send the requests to the API
embedding_1 = client.semantic_embed(request_1, model="luminous-base").embedding
embedding_2 = client.semantic_embed(request_2, model="luminous-base").embedding
embedding_question = client.semantic_embed(request_question, model="luminous-base").embedding

### 2.2 Calculating the similarity between embeddings
Now that we have embeddings for our question and our documents, we can calculate the similarity between them.
For that we use the cosine similarity. This is a measure of how similar two vectors are. The higher the value, the more similar the vectors are.

In [ ]:
# We calculate the cosine similarity between the question and the texts
similarity_1 = 1 - spatial.distance.cosine(embedding_1, embedding_question)
similarity_2 = 1 - spatial.distance.cosine(embedding_2, embedding_question)

# We print the results
print("The similarity between the question and text 1 is: " + str(similarity_1))
print("The similarity between the question and text 2 is: " + str(similarity_2))

We can see that the document with the highest similarity score is the one that we are looking for.
This semantic search is a very powerful tool to find relevant information.

### 2.3 Experiment with embeddings yourself
Now you can go ahead and experiment with embeddings yourself. 

When do they work well? 

When do they not work well?

In [ ]:
# TODO Change the text to be embedded and searched for
test_text = "A large language model (LLM) is a language model characterized by its large size. Their size is enabled by AI accelerators, which are able to process vast amounts of text data, mostly scraped from the Internet.[1] The artificial neural networks which are built can contain from tens of millions and up to billions of weights and are (pre-)trained using self-supervised learning and semi-supervised learning. Transformer architecture contributed to faster training."

# TODO Change the question to be embedded and searched for
test_question = "What is a large language model"

# run the code to embed the text and question and calculate the similarity
request_test_text = SemanticEmbeddingRequest(prompt=Prompt.from_text(test_text), representation=SemanticRepresentation.Document)
request_test_question = SemanticEmbeddingRequest(prompt=Prompt.from_text(test_question), representation=SemanticRepresentation.Query)
embedding_test_text = client.semantic_embed(request_test_text, model="luminous-base").embedding
embedding_test_question = client.semantic_embed(request_test_question, model="luminous-base").embedding
similarity_test = 1 - spatial.distance.cosine(embedding_test_text, embedding_test_question)
print("The similarity between the question and text 1 is: " + str(similarity_test))

### Step 2.4 Embedding functions
Now your task is to write two fucntions, to ease the use of embeddings in your code.

Use 128 size embeddings.

In [ ]:
def embed_query(query):
    embedding = None
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(query), representation=SemanticRepresentation.Query, compress_to_size=128)
    embedding = client.semantic_embed(request, model="luminous-base").embedding

    return embedding
    
def embed_document(document):
    embedding = None
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(document), representation=SemanticRepresentation.Document, compress_to_size=128)
    embedding = client.semantic_embed(request, model="luminous-base").embedding
    
    return embedding

def embed_symmetric(text):
    embedding = None
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Symmetric, compress_to_size=128)
    embedding = client.semantic_embed(request, model="luminous-base").embedding
    
    return embedding

embedding_query = embed_query("What is the best way to reset a password?")
embedding_document = embed_document("If a customer forgot their password, they should be able to reset it by clicking on the \"Forgot Password\" link on the login page. They can use different methods to reset their password, such as email, SMS, or security questions.")

similarity = 1 - spatial.distance.cosine(embedding_query, embedding_document)
print("The similarity between the query and the document is: " + str(similarity))

--------------------

## Step 3: Using Semantic Embeddings and Completions together to answer questions
In this section, we will use the search and completions endpoints together to answer questions.

With `semantic search`, we can find relevant information. With `completions`, we can generate text and solve tasks.


<image src="https://github.com/Aleph-Alpha/bootcamp/blob/main/img/search_strategy.png?raw=true" width="800" align="center" alt="Aleph Alpha Searching Strategy">


Our application logic is as follows:
1. We use `semantic search` to make information searchable.
2. We select the most similar document as background information.
3. We use `completions` to generate the answer to the question.

Let's use the data from the data.md file to answer some questions.

In [ ]:
# Let's use the data saved in the data.md file

with open("data.md", "r") as f:
    data = f.read().split("###")[1:]


# print the first 3 entries
for i in range(4):
    print(data[i][:100] + "...\n")

In [ ]:

question = "How can I search through my documents with embeddings?"


# Creatting the embedding for the texts and the question
request_1 = SemanticEmbeddingRequest(prompt=Prompt.from_text(text_1), representation=SemanticRepresentation.Document)
request_2 = SemanticEmbeddingRequest(prompt=Prompt.from_text(text_2), representation=SemanticRepresentation.Document)
request_question = SemanticEmbeddingRequest(prompt=Prompt.from_text(question), representation=SemanticRepresentation.Query)
embedding_1 = client.semantic_embed(request_1, model="luminous-base").embedding
embedding_2 = client.semantic_embed(request_2, model="luminous-base").embedding
embedding_question = client.semantic_embed(request_question, model="luminous-base").embedding

### Step 3.1: Using a vectordatabase to store embeddings
Instead of doin g everything ourselves, we can use a Vectordatabase to store the embeddings for us. This makes it easier to search for information.

We will be using qdrant as our vectordatabase. 
Qdrant is an open-source vectordatabase that is easy to use and fast.
You can find more information about qdrant [here](https://qdrant.tech/).

In [ ]:
# First we spin up the Qdrant server
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, Batch

q_client = QdrantClient(path="db")

q_client.recreate_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=128, distance=Distance.COSINE),
)

Now we need to store the documents in the vectordatabase

In [ ]:
# Let's create embeddings for each of the texts and store them in a list
texts = [text_1, text_2]
embeddings = []
for text in texts:
    # embed the texts
    embeddings.append(client.semantic_embed(SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Document, compress_to_size=128), model="luminous-base").embedding)
    
    
# now we can upsert the data into Qdrant
ids = list(range(len(texts)))
payloads = [{"text": text} for text in texts]

q_client.upsert(
     collection_name="test_collection",
     points=Batch(
     ids=ids,
     payloads=payloads,
     vectors=embeddings
     )
)


### Step 3.2 Using semantic search to find relevant information

Now that we have stored our documents in the vectordatabase, we can use semantic search to find relevant information.

For that we just have to send the embeddings of our question to the vectordatabase and it will return the most similar documents.

In [ ]:
# embedding the question

# We embed the question as a Query, as it is a short text
request_question = SemanticEmbeddingRequest(
    prompt=Prompt.from_text(question), 
    representation=SemanticRepresentation.Query, 
    compress_to_size=128)

embedding_question = client.semantic_embed(request_question, model="luminous-base").embedding

search_result = q_client.search(
        collection_name="test_collection",
        query_vector=embedding_question,
        limit=5, # Parameter to control the number of results
    )

for result in search_result:
    print(f"Score: {result.score}, Text: {result.payload['text']}")
    
    
# Let's select the first result to answer the question
background_text = search_result[0].payload["text"]

### Step 3.3 Using completions to generate the answer

Now that we have found the most relevant document, we can use completions to generate the answer.

We will use the same model as before. However, this time we will give it a different instruction.

TODO:
- Add additional documents to the vectordatabase
- Add additional questions
- Try to modify the prompt to get better results

In [ ]:
qa_prompt = f"""### Instruction:
{question}

### Input:
{background_text}

### Response:"""

# Let's send the prompt to the API and see what the model returns
request = CompletionRequest(
    prompt=Prompt.from_text(qa_prompt),
    maximum_tokens=100,
    temperature=0.0,
    stop_sequences=["\n"])

response = client.complete(request=request, model="luminous-supreme-control")

response_text = response.completions[0].completion

print(f"The model returned: `{response_text}`")

### Step 3.4: a search function
Now your task is to write a function that takes a question embeds it and searches for the most similar document.

Use the embedding function from step 2.4.

In [ ]:
def search(query, limit=3, threshold=0.5):
    search_result = q_client.search(
        collection_name="test_collection",
        query_vector=embed_query(query),
        limit=limit, # Parameter to control the number of results
    )
    return search_result

search("How can I search through my documents with embeddings?")

# Step 4: Classifying text with LLMs

In this section, we will use LLMs to classify text.
For that we will utilize the semantic embeddings again. 

We will use the embeddings to classify the text. This means that we will use the embeddings to find the most similar class.

While not specifially trained for classification, LLMs are able to classify text. This is because they are able to understand the meaning of text and thus, can find the most similar class.

# Step 4.1: Creating embeddings for classes
First, we need to create embeddings for our classes. 

We will use the same technique as before.
However, for classification we want to use symmetric embeddings.

In [ ]:
# TODO Define 6 different examples for smalltalk texts
class_smalltalk = ["""Hey, how are you doing?""",
"""I am doing great, how about you?""",
"""Whats up?""",
"Who are you?",
"What is your name?",
"Tell me a joke about cats.",
"Want do you think about costume parties?",]

# TODO Define 6 different examples for question texts
class_question = ["""What is the best way to reset a password?""",
"""How can I search through my documents with embeddings?""",
"What can I do with completion?",
"How does AtMan work?",
"What is he difference between symmetric and asymmetric embeddings?",]

# create a list of embeddings for each class
embeddings_smalltalk = [embed_symmetric(text) for text in class_smalltalk]
embeddings_question = [embed_symmetric(text) for text in class_question]

# Step 4.2: Classifying text
Now that we have embeddings for our classes, we can use them to classify text.

For that we just have to compute the similarity between the text embedding and the class embeddings. The class with the highest similarity score is the one that we are looking for.

In [ ]:
# Define a new text to be classified
test_text = "Hey, how are you doing?"

# Embed the test text
embedding_test_text = embed_symmetric(test_text)

# Calculate the cosine similarity between the test text and the smalltalk embeddings
similarities_smalltalk = [1 - spatial.distance.cosine(embedding_test_text, embedding) for embedding in embeddings_smalltalk]
similarities_question = [1 - spatial.distance.cosine(embedding_test_text, embedding) for embedding in embeddings_question]

# Print the results
print("The average similarity between the test text and the smalltalk embeddings is: " + str(np.mean(similarities_smalltalk)))
print("The average similarity between the test text and the question embeddings is: " + str(np.mean(similarities_question)))

# Step 4.3: Classifying text with a function
Now your task is to write a function that takes a text embeds it and classifies it.

Use the embedding function from step 2.4.


In [ ]:
# function to classify a text
def classify(text, class_names, classes):
    
    # embed the text
    embedding_text = embed_symmetric(text)
    
    # calculate the cosine similarity between the text and each class
    similarities = [np.mean([1 - spatial.distance.cosine(embedding_text, embedding) for embedding in class_]) for class_ in classes]
    # return the class with the highest similarity
    return class_names[np.argmax(similarities)]

classify("Hey, how are you doing?", ["smalltalk", "question"], [embeddings_smalltalk, embeddings_question])

### step 4.4: Visualizing the classification
Here we will visualize the classification results.

For this we will use plotly. Plotly is a powerful visualization library that allows us to create interactive plots.

In [ ]:
# reducing the size of the embeddings to 2 dimensions for visualization
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

pca.fit(embeddings_smalltalk + embeddings_question)

embeddings_smalltalk_2d = pca.transform(embeddings_smalltalk)
embeddings_question_2d = pca.transform(embeddings_question)
embedding_test_text_2d = pca.transform([embedding_test_text])

# Plotting the embeddings
import plotly.express as px

# create a generic figure
fig = px.scatter()

# add the embeddings to the figure
fig.add_scatter(x=embeddings_smalltalk_2d[:,0], y=embeddings_smalltalk_2d[:,1], mode="markers", name="smalltalk")
fig.add_scatter(x=embeddings_question_2d[:,0], y=embeddings_question_2d[:,1], mode="markers", name="question")
fig.add_scatter(x=embedding_test_text_2d[:,0], y=embedding_test_text_2d[:,1], mode="markers", name="test text")

fig.show()

In [ ]:
classes = [
    {
        "name" : "Landwirtschaft, Jagd und damit verbundene Tätigkeiten",
        "sub_classes" : [
            {
                "name" : "Anbau einjähriger Pflanzen",
                "sub_classes" : [
                    {
                        "name" : "Anbau von Getreide (ohne Reis), Hülsenfrüchten und Ölsaaten ",
                        "sub_classes" : ""
                    },
                    {
                        "name" : "Anbau von ohne Reis",
                        "sub_classes" : ""
                    }
                ]
             },
            {
                "name" : "Betrieb von Baumschulen sowie Anbau von Pflanzen zu Vermehrungszwecken ",
                "sub_classes" : [
                    {
                        "name" : "Anbau von Zimmerpflanzen, Beet- und Balkonpflanzen",
                        "sub_classes" : ""
                    },
                    {
                        "name" : "Betrieb von Baumschulen sowie Anbau von Pflanzen zu Vermehrungszwecken ",
                        "sub_classes" : ""
                    }
                ]
             }
        ]
    },
    {
        "name" : "Kohlenbergbau",
        "sub_classes" : [
            {
                "name" : "Steinkohlenbergbau",
                "sub_classes" : [
                    {
                        "name" : "Steinkohlenbergbau",
                        "sub_classes" : ""
                    }
                ]
             },
            {
                "name" : "Braunkohlenbergbau ",
                "sub_classes" : [
                    {
                        "name" : "Braunkohlenbergbau",
                        "sub_classes" : ""
                    }
                ]
             }
        ]
    }
]

In [ ]:
# go through each level of the classes and compare their embeddings for similarity

text_to_compare = "Ich habe einen Bauernhof, der Getreide anbaut"

compare_embedding = embed_symmetric(text_to_compare)

class_similarities = [1 - spatial.distance.cosine(compare_embedding, embed_symmetric(class_["name"])) for class_ in classes]

print("The similarity between the text and the classes is: " + str(class_similarities))

# if the class has sub classes, we compare the text to the sub classes

determined_class = classes[np.argmax(class_similarities)]

if determined_class["sub_classes"] != "":
    
    sub_class_similarities = [1 - spatial.distance.cosine(compare_embedding, embed_symmetric(sub_class["name"])) for sub_class in determined_class["sub_classes"]]
    
    print("The similarity between the text and the sub classes is: " + str(sub_class_similarities))
    
    determined_sub_class = determined_class["sub_classes"][np.argmax(sub_class_similarities)]
    
    print("The determined class is: " + determined_sub_class["name"])
    
    # if the sub class has sub classes, we compare the text to the sub classes
    
    if determined_sub_class["sub_classes"] != "":

        sub_sub_class_similarities = [1 - spatial.distance.cosine(compare_embedding, embed_symmetric(sub_sub_class["name"])) for sub_sub_class in determined_sub_class["sub_classes"]]

        print("The similarity between the text and the sub sub classes is: " + str(sub_sub_class_similarities))

        determined_sub_sub_class = determined_sub_class["sub_classes"][np.argmax(sub_sub_class_similarities)]

        print("The determined class is: " + determined_sub_sub_class["name"])

In [ ]:
# now in a more general way

class_similarities = [1 - spatial.distance.cosine(compare_embedding, embed_symmetric(class_["name"])) for class_ in classes]

determined_class = classes[np.argmax(class_similarities)]

while determined_class["sub_classes"] != "":
    
    sub_class_similarities = [1 - spatial.distance.cosine(compare_embedding, embed_symmetric(sub_class["name"])) for sub_class in determined_class["sub_classes"]]
    
    determined_sub_class = determined_class["sub_classes"][np.argmax(sub_class_similarities)]
    
    determined_class = determined_sub_class
    
print("The determined class is: " + determined_class["name"])

--------------------

## Step 5: AtMan: Understanding the model's decisions
This section will show you how to use AtMan to understand the model's decisions.

With our `explain`-endpoint you can get an explanation of the model's output. In more detail, we return how much the log-probabilites of the already generated completion would change if we supress indivdual parts (based on the granularity you chose) of a prompt. Please refer to this part of our documentation if you would like to know more about our explainability method in general.

### Step 5.1: Using AtMan to understand the model's decisions
Now that we have seen how to generate text, we want to be able to understand the model's decisions.

For that we will use AtMan. AtMan is our explainability tool that allows us to understand the model's decisions.

By surpressing individual parts of the prompt, we can see how much the model's output changes. This allows us to understand which parts of the prompt are important for the model's decision.

In [ ]:
text = "The quick brown fox jumps over the lazy dog.\nThe color of the fox is"
# Here we define a TextControl that will be used to control the attention on the prompt.


# Change the factor to 0.0 to see what happens.
control = TextControl(start=10, length=5, factor=0.0)

# Prompt without control
prompt = Prompt.from_text(text)

# Prompt with control
prompt2 = Prompt.from_text(text, controls=[control])

request = CompletionRequest(prompt=prompt, maximum_tokens=10, stop_sequences=["."], log_probs=5)
request2 = CompletionRequest(prompt=prompt2, maximum_tokens=10, stop_sequences=["."], log_probs=5)
result = client.complete(request = request, model="luminous-extended")
result2 = client.complete(request = request2, model="luminous-extended")

print(f" The most probable completion 1 is : ", result.completions[0].completion)
print(f"The probability of brown in completion 1 is {result.completions[0].log_probs[0][' brown']}")

print(f" The most probable completion 2 is : ", result2.completions[0].completion)
print(f"The probability of brown in completion 2 is {result2.completions[0].log_probs[0][' brown']}")

### Step 5.2: Using AtMan via ExplainRequest
Now that we have seen how to use AtMan, let's see how we can use it via the API.

For that we will use the `ExplanationRequest`. This request allows us to get an explanation for a completion.

    

In [ ]:
prompt_text = """Answer the question based on the context.

Context: According to tradition, on April 21, 753 BC, Romulus and his twin brother Remus founded Rome in the place where they had been suckled as orphans by a she-wolf.

Q: In which month was Rome founded?

A:"""

params = {
    "prompt": Prompt.from_text(prompt_text),
    "maximum_tokens": 1,
}
request = CompletionRequest(**params)
response = client.complete(request=request, model="luminous-supreme")
completion = response.completions[0].completion

exp_req = ExplanationRequest(Prompt.from_text(prompt_text), completion, prompt_granularity="paragraph")
response_explain = client.explain(exp_req, model="luminous-supreme")

explanations = response_explain[1][0].items[0][0]

for item in explanations:
    start = item.start
    end = item.start + item.length
    print(f"""EXPLAINED TEXT: {prompt_text[start:end]}
Score: {np.round(item.score, decimals=3)}""")

As you can see from the example. The explanation helps us locate the relevant information that Luminous used.
Please keep in mind, that especially the control models will have a very high explainability on the instructions. This is because they are trained to solve specific tasks. This means that they will always use the same parts of the instructions to solve the task.

This can be easily managed by only looking at the explainability of the input. This will give us a better understanding of what the model is doing.


## Step 6: Chaining multiple requests to solve complex tasks
Sometimes, we need to solve complex tasks. For that, we can chain multiple requests together.

Similar to Humans, LLMs produce more robust results if they are able to solve a task in multiple steps. This is because they can focus on one task at a time and do not have to solve everything at once (end-to-end).

While solving tasks end-to-end may be very convenient, it is not always the best solution. This is because the model may not be able to focus on the most important parts of the task. This can lead to worse results.

It is also much more difficult to debug and understand what the model is doing. This is because the model is solving the task in one step and we cannot see what it is doing.

In this example, we will combine the techniques that we have learned so far to solve a complex task.

### Step 6.1: writing a functional chatbot

Let's combine all the techniques that we have learned so far to write a functional chatbot.

In [ ]:
history = []
# function that enables chat

def chat(message, history):
    
    # first, find out if the message is a question or smalltalk
    classification = classify(message, ["smalltalk", "question"], [embeddings_smalltalk, embeddings_question])
    
    # if the message is smalltalk, use the smalltalk function
    
    if classification == "smalltalk":
        print("running smalltalk")
        response = smalltalk(message, history)
        
    # if the message is a question, use the chat_answer function
    elif classification == "question":
        print("running chat_answer")
        
        background_information = search(message, limit=1)[0].payload["text"]
        
        response = qa_answer(message, history, background_information)
        
    # add the message to the history    
    history.append("User: " + message)
    history.append("AlphaBot: " + response)
    
    # return the response
    return response, history

In [ ]:
print(chat("What do you know about speedboats?", history))

### Step 6.2: putting everything in a visual interface

In [ ]:
import gradio as gr
import random
import time

history = []

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        global history
        bot_message, history = chat(message, history)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    
demo.launch()


---------------
## Conclusion
In this tutorial, we have seen how to use our API to generate text, search for information, and solve tasks.

We have also seen how to chain multiple requests together to solve complex tasks.

We hope that this tutorial was helpful to you. If you have any questions, please do not hesitate ask us.